In [46]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
            SparkSession.builder
                .appName("SparkPractice")
                .master("local[4]")
                .config("spark.dynamicAllocation.enabled","false")
                .config("spark.sql.adaptive.enabled","false") 
                .getOrCreate()
)
sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=SparkPractice>

In [47]:
df = spark.read.csv("aadhaar_data.csv",header = True)
df.show(5)

+--------------------+--------------------+----------------+---------------+-----------+-------+------+---+----------------+-----------------+-----------------------+------------------------------+
|           Registrar|     EnrolmentAgency|           State|       District|SubDistrict|PinCode|Gender|Age|Aadhaargenerated|EnrolmentRejected|Residentsprovidingemail|Residentsprovidingmobilenumber|
+--------------------+--------------------+----------------+---------------+-----------+-------+------+---+----------------+-----------------+-----------------------+------------------------------+
| Chief Registrar ...|District Registra...|Himachal Pradesh|Lahul and Spiti|  Lahul (t)| 175132|     M|  0|               1|                0|                      0|                             1|
| Chief Registrar ...|District Registra...|Himachal Pradesh|       Hamirpur|Dhatwal(st)| 176040|     M|  0|               1|                0|                      0|                             0|
| Chief Re

In [48]:
#Count the Male Residents and Female Resident in each state who have enrolled in Aadhaar

#Method
#1) Filter by Gender i.e Male or Female
#2) Filter the Male values 
#3) Group By state
#4) And Count the gender and rename as male count

print("Male Count in each state is :")
print("\n\n")
df1 = df.select(df.State,df.Gender,when(df.Gender == 'M',df.Gender).alias('MaleCount'))
df1.filter(df1.MaleCount == 'M').groupBy('State').agg(count('Gender').alias('MaleCount')).show()

print("\n\n")
print("Female Count in each state is :")
print("\n\n")
df1 = df.select(df.State,df.Gender,when(df.Gender == 'F',df.Gender).alias('MaleCount'))
df1.filter(df1.MaleCount == 'F').groupBy('State').agg(count('Gender').alias('MaleCount')).show()

Male Count in each state is :



+--------------------+---------+
|               State|MaleCount|
+--------------------+---------+
|            Nagaland|      226|
|           Karnataka|     2764|
|              Odisha|     3456|
|              Kerala|     2020|
|          Tamil Nadu|     3066|
|        Chhattisgarh|      786|
|      Andhra Pradesh|      945|
|         Lakshadweep|        6|
|      Madhya Pradesh|     3203|
|              Punjab|     1750|
|             Manipur|      519|
|                 Goa|       88|
|             Mizoram|      246|
|Dadra and Nagar H...|       13|
|    Himachal Pradesh|      330|
|          Puducherry|       41|
|   Jammu and Kashmir|     1484|
|             Haryana|      974|
|           Jharkhand|     1092|
|   Arunachal Pradesh|      364|
+--------------------+---------+
only showing top 20 rows




Female Count in each state is :



+--------------------+---------+
|               State|MaleCount|
+--------------------+---------+
|           

In [59]:
#2 Count the rejected residents in each state
#Method
#1) Filter by enrolment rejected
#2) Remove the null values
#3) Group BY state
#4) Perform count on it
print("Count of rejected residents in each state is")
print('\n\n')
df1 = df.select(df.State,df.EnrolmentRejected,when(df.EnrolmentRejected == 1,df.EnrolmentRejected)
                .alias('Rejected'))

df1.filter(df1.Rejected == 1).groupBy('State').agg(count('Rejected').alias("Rejected")).show()

Count of rejected residents in each state is



+--------------------+--------+
|               State|Rejected|
+--------------------+--------+
|            Nagaland|      64|
|           Karnataka|     746|
|              Odisha|    1938|
|              Kerala|     355|
|          Tamil Nadu|     894|
|        Chhattisgarh|     339|
|      Andhra Pradesh|     314|
|      Madhya Pradesh|    1358|
|              Punjab|     878|
|             Manipur|     154|
|                 Goa|      27|
|             Mizoram|     126|
|Dadra and Nagar H...|      14|
|    Himachal Pradesh|     108|
|          Puducherry|      10|
|             Haryana|     362|
|   Jammu and Kashmir|     924|
|           Jharkhand|     482|
|   Arunachal Pradesh|     112|
|             Gujarat|    1478|
+--------------------+--------+
only showing top 20 rows



In [62]:
#3 Count the rejected residents in each state and then calculate the average of the rejected
#residents in India (You have to write two MR jobs that takes output of first job as input to
#the second job)

print("Count of rejected residents in each state is")
print('\n\n')
df1 = df.select(df.State,df.EnrolmentRejected,when(df.EnrolmentRejected == 1,df.EnrolmentRejected)
                .alias('Rejected'))

df1.filter(df1.Rejected == 1).groupBy('State').agg(count('Rejected').alias("Rejected")).show()

print("The average of rejected members across India is ")
print("\n\n")
df2 = df1.agg(mean('Rejected').alias("AvgRejected")).show()

Count of rejected residents in each state is



+--------------------+--------+
|               State|Rejected|
+--------------------+--------+
|            Nagaland|      64|
|           Karnataka|     746|
|              Odisha|    1938|
|              Kerala|     355|
|          Tamil Nadu|     894|
|        Chhattisgarh|     339|
|      Andhra Pradesh|     314|
|      Madhya Pradesh|    1358|
|              Punjab|     878|
|             Manipur|     154|
|                 Goa|      27|
|             Mizoram|     126|
|Dadra and Nagar H...|      14|
|    Himachal Pradesh|     108|
|          Puducherry|      10|
|             Haryana|     362|
|   Jammu and Kashmir|     924|
|           Jharkhand|     482|
|   Arunachal Pradesh|     112|
|             Gujarat|    1478|
+--------------------+--------+
only showing top 20 rows

The average of rejected members across India is 



+-----------+
|AvgRejected|
+-----------+
|        1.0|
+-----------+

